In [90]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def load_image(image_path):
    image = Image.open(image_path)
    image = image.resize((160, 160)) 
    image = np.asarray(image)
    image = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    image = np.expand_dims(image, axis=0)
    return image

def load_pb_model(model_filepath):
    # Load TensorFlow Graph from .pb file
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()

    with open(model_filepath, "rb") as f:
        graph_def.ParseFromString(f.read())
    
    with graph.as_default():
        tf.import_graph_def(graph_def, name="")

    return graph

def generate_embeddings(graph, dataset_path):
    embeddings = []
    labels = []
    with tf.compat.v1.Session(graph=graph) as sess:
        for person_name in os.listdir(dataset_path):
            person_path = os.path.join(dataset_path, person_name)
            if os.path.isdir(person_path):  # Add this check here
                for image_name in os.listdir(person_path):
                    image_path = os.path.join(person_path, image_name)
                    if os.path.isfile(image_path):  # Optionally, ensure this is a file
                        image = load_image(image_path)
                        
                        # Fetch the tensors
                        images_placeholder = graph.get_tensor_by_name("input:0")
                        embeddings_tensor = graph.get_tensor_by_name("embeddings:0")
                        phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")

                        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
                        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
                        
                        embeddings.append(embedding)
                        labels.append(person_name)
    embeddings = np.asarray(embeddings).reshape(len(embeddings), -1)  # Reshape to (num_samples, embedding_size)
    labels = np.asarray(labels)
    return embeddings, labels


model_filepath = '20180402-114759/20180402-114759.pb'  
graph = load_pb_model(model_filepath)
dataset_path = 'dataset'  
embeddings, labels = generate_embeddings(graph, dataset_path)


In [91]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

def preprocess_image(image_path, target_size=(160, 160)):
    """Load an image file, resize it to target size, and normalize it."""
    image = Image.open(image_path)
    image = image.resize(target_size)
    image_array = np.array(image)
    # Normalize the image
    image_array = image_array.astype('float32')
    mean, std = image_array.mean(), image_array.std()
    image_array = (image_array - mean) / std
    return image_array

def load_dataset(dataset_path, target_size=(160, 160)):
    """Load and preprocess the entire dataset."""
    X, y = [], []
    # Walk through the person directories and list their images
    for person_name in tqdm(os.listdir(dataset_path), desc="Processing"):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue  # Skip non-directory files
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            if not os.path.isfile(image_path) or not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue  # Skip non-image files
            image_array = preprocess_image(image_path, target_size)
            X.append(image_array)
            y.append(person_name)
    return np.array(X), np.array(y)

# Path to your dataset
dataset_path = 'dataset'

# Load and preprocess the dataset
X, y = load_dataset(dataset_path)

print(f"Loaded and preprocessed {len(X)} images.")


Processing: 100%|██████████| 5749/5749 [00:26<00:00, 214.43it/s]


Loaded and preprocessed 13232 images.


In [92]:
# Encoding labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

def find_nearest_neighbor(test_embedding, embeddings):
    min_distance = np.inf
    nearest_neighbor_idx = -1
    for idx, embedding in enumerate(embeddings):
        distance = euclidean(test_embedding, embedding)
        if distance < min_distance:
            min_distance = distance
            nearest_neighbor_idx = idx
    return nearest_neighbor_idx

In [93]:
# Function to get an embedding for a single image using FaceNet
def get_embedding(model, image_path):
    image = load_image(image_path)
    with tf.compat.v1.Session(graph=model) as sess:
        images_placeholder = model.get_tensor_by_name("input:0")
        embeddings_tensor = model.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = model.get_tensor_by_name("phase_train:0")
        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)
    return embedding.flatten()

In [101]:
# Predicting person for a test image
test_image_path = 'test/Bob_Curtis/Bob_Curtis_0001.jpg'
test_embedding = get_embedding(facenet_model, test_image_path)
nearest_neighbor_idx = find_nearest_neighbor(test_embedding, embeddings)

# Assuming nearest_neighbor_idx is valid
predicted_label = labels_encoded[nearest_neighbor_idx]
predicted_person = label_encoder.inverse_transform([predicted_label])

print(f"Predicted person: {predicted_person[0]}")

Predicted person: Bob_Curtis


In [31]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam


# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_encoded, test_size=0.2, random_state=42)

# Define the new model architecture
embedding_input_size = embeddings.shape[1] 
input_layer = Input(shape=(embedding_input_size,))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.5)(x)
output_layer = Dense(len(np.unique(labels_encoded)), activation='softmax')(x)

# Define the model
classifier_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
classifier_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = classifier_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)



Epoch 1/10
331/331 [==============================] - 2s 5ms/step - loss: 7.9440 - accuracy: 0.0500 - val_loss: 7.7655 - val_accuracy: 0.0514
Epoch 2/10
331/331 [==============================] - 2s 5ms/step - loss: 7.0540 - accuracy: 0.0855 - val_loss: 7.6775 - val_accuracy: 0.1050
Epoch 3/10
331/331 [==============================] - 2s 5ms/step - loss: 6.6132 - accuracy: 0.1188 - val_loss: 7.6428 - val_accuracy: 0.1292
Epoch 4/10
331/331 [==============================] - 2s 5ms/step - loss: 6.1648 - accuracy: 0.1459 - val_loss: 7.6387 - val_accuracy: 0.1519
Epoch 5/10
331/331 [==============================] - 1s 5ms/step - loss: 5.7109 - accuracy: 0.1780 - val_loss: 7.6379 - val_accuracy: 0.1810
Epoch 6/10
331/331 [==============================] - 2s 5ms/step - loss: 5.2242 - accuracy: 0.2133 - val_loss: 7.6460 - val_accuracy: 0.2134
Epoch 7/10
331/331 [==============================] - 2s 5ms/step - loss: 4.6877 - accuracy: 0.2550 - val_loss: 7.6501 - val_accuracy: 0.2388
Epoch 

In [32]:
# Evaluate the model on the test set
test_loss, test_accuracy = classifier_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


83/83 [==============================] - 0s 3ms/step - loss: 7.8822 - accuracy: 0.2928
Test Accuracy: 29.28%


In [ ]:
# Save the model
model_save_path = 'path_to_save_your_model/facenet_classifier.h5'
classifier_model.save(model_save_path)

# To save just the weights
weights_save_path = 'path_to_save_your_model/facenet_classifier_weights.h5'
classifier_model.save_weights(weights_save_path)


In [ ]:
def predict_person(image_path, facenet_model, classifier_model, label_encoder):
    # Load and preprocess the image
    image = load_image(image_path)  # This should be the same function you used before

    # Generate embedding using FaceNet
    with tf.compat.v1.Session(graph=facenet_model) as sess:
        images_placeholder = facenet_model.get_tensor_by_name("input:0")
        embeddings_tensor = facenet_model.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = facenet_model.get_tensor_by_name("phase_train:0")
        
        feed_dict = {images_placeholder: image, phase_train_placeholder: False}
        embedding = sess.run(embeddings_tensor, feed_dict=feed_dict)

    embedding = np.reshape(embedding, (1, -1))  # Reshape the embedding to match the classifier's input expectation
    
    # Predict using the classifier model
    prediction = classifier_model.predict(embedding)
    predicted_label_idx = np.argmax(prediction, axis=1)
    predicted_label = label_encoder.inverse_transform(predicted_label_idx)
    
    return predicted_label

# Example usage
image_path = 'path_to_your_new_image.jpg'
facenet_graph = load_pb_model('20180402-114759/20180402-114759.pb')  # Load the FaceNet model as before
predicted_person = predict_person(image_path, facenet_graph, classifier_model, label_encoder)
print(f"Predicted person: {predicted_person}")
